In [1]:
import os
os.listdir()


['.ipynb_checkpoints',
 '0I6U0N56BVTVGY24EM2FARBNIC.m3u8_in=5ca307641ef252bfd06e3224bedaa7e1__36b885d9f3c4f5ff1fdf59bc906367c9__1747474221__ni',
 '22-47_U1_IDS-3[1].pptx',
 '23891A67B2 (2).pdf',
 '23891A67B2.c',
 '23891A67B2.pdf',
 '2pics.docx',
 '2pics.pdf',
 '3.Employee Management System – Sort Employees by Salary using Quick Sort.pptx',
 '3.Employee Management System – Sort Employees by Salary using Quick Sort[1].pptx',
 '35afbd3360e0696aa99a45febea95f28.jpg',
 '5 Sports-Tournament-Scheduler-Efficient-Match-Scheduling-Using-Divide-and-Conquer[2].pptx',
 '79-ADA[1].pptx',
 '8033_3_8019429_1748001786_AWS Course Completion Certificate.pdf',
 '947037.jpg',
 'A2 De unit 5 Project Report.docx.docx',
 'Aadhaar.jpg',
 'ADA-A3[1][1].pptx',
 'AI_Agents_Report_PageNumbers_Centered.docx',
 'AI_Powered_Collections_Strategy.pptx',
 'Akula_Anjana_Internship_report[1].docx',
 'allotment.pdf',
 'Anaconda3-2025.06-0-Windows-x86_64.exe',
 'apache-tomcat-9.0.104.exe',
 'app.py',
 'archive',
 'archive (

In [2]:
import pandas as pd
import sqlite3


In [3]:
orders = pd.read_csv("orders.csv")
orders.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [4]:
users = pd.read_json("users.json")
users.head()


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [8]:
conn = sqlite3.connect("restaurants.db")
cursor = conn.cursor()

# Drop table if it already exists
cursor.execute("DROP TABLE IF EXISTS restaurants")

with open("restaurants.sql", "r", encoding="utf-8") as f:
    sql_script = f.read()

cursor.executescript(sql_script)
conn.commit()


In [9]:
restaurants = pd.read_sql_query(
    "SELECT * FROM restaurants", conn
)

restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [10]:
orders_users = pd.merge(
    orders,
    users,
    on="user_id",
    how="left"
)

final_df = pd.merge(
    orders_users,
    restaurants,
    on="restaurant_id",
    how="left"
)

final_df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [12]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)


In [18]:
final_df.columns


Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating'],
      dtype='object')

In [20]:
list(final_df.columns)


['order_id',
 'user_id',
 'restaurant_id',
 'order_date',
 'total_amount',
 'restaurant_name_x',
 'name',
 'city',
 'membership',
 'restaurant_name_y',
 'cuisine',
 'rating']

In [21]:
final_df[final_df['membership'] == 'Gold'] \
    .groupby('city')['total_amount'] \
    .sum() \
    .sort_values(ascending=False)


city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

In [22]:
MEMBERSHIP_COL = 'membership'
AMOUNT_COL = 'total_amount'
CITY_COL = 'city'
USER_COL = 'user_id'
CUISINE_COL = 'cuisine'
RATING_COL = 'rating'
RESTAURANT_COL = 'restaurant_name'
DATE_COL = 'order_date'


In [23]:
final_df.groupby(CUISINE_COL)[AMOUNT_COL] \
    .mean() \
    .sort_values(ascending=False)


cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

In [24]:
final_df.groupby(USER_COL)[AMOUNT_COL] \
    .sum() \
    .loc[lambda x: x > 1000] \
    .count()


np.int64(2544)

In [25]:
final_df['rating_range'] = pd.cut(
    final_df[RATING_COL],
    bins=[3.0, 3.5, 4.0, 4.5, 5.0]
)

final_df.groupby('rating_range')[AMOUNT_COL] \
    .sum() \
    .sort_values(ascending=False)


C:\Users\rpran\AppData\Local\Temp\ipykernel_14536\2693199462.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  final_df.groupby('rating_range')[AMOUNT_COL] \


rating_range
(4.5, 5.0]    2197030.75
(4.0, 4.5]    1960326.26
(3.0, 3.5]    1881754.57
(3.5, 4.0]    1717494.41
Name: total_amount, dtype: float64

In [26]:
final_df[final_df[MEMBERSHIP_COL] == 'Gold'] \
    .groupby(CITY_COL)[AMOUNT_COL] \
    .mean() \
    .sort_values(ascending=False)


city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

In [27]:
final_df.groupby(CUISINE_COL).agg(
    total_revenue=(AMOUNT_COL, 'sum'),
    restaurant_count=('restaurant_id', 'nunique')
).sort_values(['restaurant_count', 'total_revenue'],
              ascending=[True, False])


,total_revenue,restaurant_count
cuisine,,
Chinese,1930504.65,120
Italian,2024203.80,126
Indian,1971412.58,126
Mexican,2085503.09,128


In [30]:
list(final_df.columns)


['order_id',
 'user_id',
 'restaurant_id',
 'order_date',
 'total_amount',
 'restaurant_name_x',
 'name',
 'city',
 'membership',
 'restaurant_name_y',
 'cuisine',
 'rating',
 'rating_range']

In [33]:
final_df[final_df[MEMBERSHIP_COL] == 'Gold'].shape[0]


4987

In [28]:
round(
    (final_df[final_df[MEMBERSHIP_COL] == 'Gold'].shape[0]
     / final_df.shape[0]) * 100
)


50

In [34]:
round(
    final_df[final_df[CITY_COL] == 'Hyderabad'][AMOUNT_COL].sum()
)


1889367

In [35]:
final_df[USER_COL].nunique()


2883

In [36]:
final_df[final_df[RATING_COL] >= 4.5].shape[0]


3374

In [37]:
top_city = final_df[final_df[MEMBERSHIP_COL] == 'Gold'] \
    .groupby(CITY_COL)[AMOUNT_COL] \
    .sum() \
    .idxmax()

final_df[
    (final_df[MEMBERSHIP_COL] == 'Gold') &
    (final_df[CITY_COL] == top_city)
].shape[0]


1337

In [38]:
round(
    final_df[final_df[MEMBERSHIP_COL] == 'Gold'][AMOUNT_COL].mean(),
    2
)


np.float64(797.15)